# Install and init DVC

Prerequisites: 
-  DVC and requirements.txt packages installed (if not - check README.md file for instructions)
-  A project repository is a Git repo 



## Checkout branch `tutorial`

In [1]:
!git checkout -b dvc-tutorial

M	dvc-3-automate-experiments.ipynb
Переключено на новую ветку «dvc-tutorial»


## Initialize DVC

References: 
- https://dvc.org/doc/get-started/initialize 

In [2]:
!dvc init


You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


## Commit changes

In [3]:
%%bash

git add .
git commit -m "Initialize DVC"

[dvc-tutorial 817567b] Initialize DVC
 8 files changed, 131 insertions(+), 114 deletions(-)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvc/plots/confusion.json
 create mode 100644 .dvc/plots/default.json
 create mode 100644 .dvc/plots/scatter.json
 create mode 100644 .dvc/plots/smooth.json
 create mode 100644 .dvcignore


# Build automated pipelines

## Create `data_load` stage


In [4]:
!mkdir -p data

In [5]:
!dvc run -n data_load \
    -d src/data_load.py \
    -o data/iris.csv \
    -o data/classes.json \
    -p data_load \
    python src/data_load.py \
        --config=params.yaml

Running stage 'data_load' with command:                               core>
	python src/data_load.py --config=params.yaml
Creating 'dvc.yaml'                                                   core>
Adding stage 'data_load' in 'dvc.yaml'
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock


In [6]:
%%bash

du -sh data/*

4,0K	data/iris.csv


In [7]:
!tree -I dvc-venv

.
├── data
│   └── iris.csv
├── dvc-3-automate-experiments.ipynb
├── dvc.lock
├── dvc.yaml
├── params.yaml
├── README.md
├── requirements.txt
└── src
    ├── data_load.py
    ├── evaluate.py
    ├── featurization.py
    ├── __init__.py
    ├── split_dataset.py
    └── train.py

2 directories, 13 files


## dvc.yaml

In [8]:
!cat dvc.yaml

stages:
  data_load:
    cmd: python src/data_load.py --config=params.yaml
    deps:
    - src/data_load.py
    params:
    - data_load
    outs:
    - data/classes.json
    - data/iris.csv


## params.yaml

In [9]:
!cat params.yaml


data_load:
  raw_data_path: data/iris.csv
  classes_names_path: data/classes.json

featurize:
  features_path: data/iris_featurized.csv
  target_column: target


data_split:
  test_size: 0.2
  train_path: data/train.csv
  test_path: data/test.csv


train:
  model_path: data/model.joblib


evaluate:
  metrics_file: data/metrics.json
  confusion_matrix: data/cm.csv


## Reproduce a pipeline

In [10]:
!dvc repro

Stage 'data_load' is cached - skipping run, checking out outputs      core>


## Change params.yaml and reproduce 

Add a new line into `data_load` section:
    `dummy_param: dummy_value`

In [11]:
!dvc repro

Running stage 'data_load' with command:                               core>
	python src/data_load.py --config=params.yaml
Updating lock file 'dvc.lock'                                         core>

To track the changes with git, run:

	git add dvc.lock


# Build end-to-end Machine Learning pipeline
Stages 
- extract features 
- split dataset 
- train 
- evaluate 


## Add feature extraction stage

In [12]:
!dvc run -n feature_extraction \
    -d src/featurization.py \
    -d data/iris.csv \
    -o data/iris_featurized.csv \
    -p data_load,featurize \
    python src/featurization.py \
        --config=params.yaml

Running stage 'feature_extraction' with command:                      core>
	python src/featurization.py --config=params.yaml
Adding stage 'feature_extraction' in 'dvc.yaml'                       core>
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml


In [13]:
!ls 

data				  dvc.lock  dvc.yaml	 README.md	   src
dvc-3-automate-experiments.ipynb  dvc-venv  params.yaml  requirements.txt


In [14]:
!cat dvc.yaml

stages:
  data_load:
    cmd: python src/data_load.py --config=params.yaml
    deps:
    - src/data_load.py
    params:
    - data_load
    outs:
    - data/classes.json
    - data/iris.csv
  feature_extraction:
    cmd: python src/featurization.py --config=params.yaml
    deps:
    - data/iris.csv
    - src/featurization.py
    params:
    - data_load
    - featurize
    outs:
    - data/iris_featurized.csv


In [15]:
import pandas as pd

features = pd.read_csv('data/iris_featurized.csv')
features.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [16]:
!git status -s

 M dvc-3-automate-experiments.ipynb
 M params.yaml
?? dvc.lock
?? dvc.yaml


In [17]:
%%bash
git add .
git commit -m "Add stage features_extraction"

[dvc-tutorial cc97e57] Add stage features_extraction
 4 files changed, 100 insertions(+), 51 deletions(-)
 create mode 100644 dvc.lock
 create mode 100644 dvc.yaml


## Add split train/test stage

In [18]:
!dvc run -n split_dataset \
    -d src/split_dataset.py \
    -d data/iris_featurized.csv \
    -o data/train.csv \
    -o data/test.csv \
    -p featurize,data_split \
        python src/split_dataset.py \
            --config=params.yaml

Running stage 'split_dataset' with command:                           core>
	python src/split_dataset.py --config=params.yaml
Adding stage 'split_dataset' in 'dvc.yaml'                            core>
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml


In [19]:
!cat dvc.yaml

stages:
  data_load:
    cmd: python src/data_load.py --config=params.yaml
    deps:
    - src/data_load.py
    params:
    - data_load
    outs:
    - data/classes.json
    - data/iris.csv
  feature_extraction:
    cmd: python src/featurization.py --config=params.yaml
    deps:
    - data/iris.csv
    - src/featurization.py
    params:
    - data_load
    - featurize
    outs:
    - data/iris_featurized.csv
  split_dataset:
    cmd: python src/split_dataset.py --config=params.yaml
    deps:
    - data/iris_featurized.csv
    - src/split_dataset.py
    params:
    - data_split
    - featurize
    outs:
    - data/test.csv
    - data/train.csv


In [20]:
%%bash
git add .
git commit -m "Add stage split_dataset"

[dvc-tutorial 7f5429e] Add stage split_dataset
 2 files changed, 32 insertions(+)


## Add train stage

In [21]:
!dvc run -n train \
    -d src/train.py \
    -d data/train.csv \
    -o data/model.joblib \
    -p data_split,train \
        python src/train.py \
            --config=params.yaml

Running stage 'train' with command:                                   core>
	python src/train.py --config=params.yaml
Adding stage 'train' in 'dvc.yaml'                                    core>
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml


In [22]:
!cat dvc.yaml

stages:
  data_load:
    cmd: python src/data_load.py --config=params.yaml
    deps:
    - src/data_load.py
    params:
    - data_load
    outs:
    - data/classes.json
    - data/iris.csv
  feature_extraction:
    cmd: python src/featurization.py --config=params.yaml
    deps:
    - data/iris.csv
    - src/featurization.py
    params:
    - data_load
    - featurize
    outs:
    - data/iris_featurized.csv
  split_dataset:
    cmd: python src/split_dataset.py --config=params.yaml
    deps:
    - data/iris_featurized.csv
    - src/split_dataset.py
    params:
    - data_split
    - featurize
    outs:
    - data/test.csv
    - data/train.csv
  train:
    cmd: python src/train.py --config=params.yaml
    deps:
    - data/train.csv
    - src/train.py
    params:
    - data_split
    - train
    outs:
    - data/model.joblib


In [23]:
%%bash
git add .
git commit -m "Add stage train"

[dvc-tutorial a6ba55d] Add stage train
 2 files changed, 28 insertions(+)


## Add evaluate stage

In [24]:
!dvc run -n evaluate \
    -d src/evaluate.py \
    -d data/test.csv \
    -d data/model.joblib \
    -d data/classes.json \
    -m data/metrics.json \
    --plots data/cm.csv \
    -p data_load,data_split,train,evaluate \
        python src/evaluate.py \
            --config=params.yaml

Running stage 'evaluate' with command:                                core>
	python src/evaluate.py --config=params.yaml
Adding stage 'evaluate' in 'dvc.yaml'                                 core>
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml


In [25]:
!cat dvc.yaml

stages:
  data_load:
    cmd: python src/data_load.py --config=params.yaml
    deps:
    - src/data_load.py
    params:
    - data_load
    outs:
    - data/classes.json
    - data/iris.csv
  feature_extraction:
    cmd: python src/featurization.py --config=params.yaml
    deps:
    - data/iris.csv
    - src/featurization.py
    params:
    - data_load
    - featurize
    outs:
    - data/iris_featurized.csv
  split_dataset:
    cmd: python src/split_dataset.py --config=params.yaml
    deps:
    - data/iris_featurized.csv
    - src/split_dataset.py
    params:
    - data_split
    - featurize
    outs:
    - data/test.csv
    - data/train.csv
  train:
    cmd: python src/train.py --config=params.yaml
    deps:
    - data/train.csv
    - src/train.py
    params:
    - data_split
    - train
    outs:
    - data/model.joblib
  evaluate:
    cmd: python src/evaluate.py --config=params.yaml
    deps:
    - data/classes.json
    - data/model.joblib
    - data/test.csv
    - src/evaluate.py


In [26]:
%%bash
git add .
git commit -m "Add stage evaluate"

[dvc-tutorial 5138317] Add stage evaluate
 2 files changed, 47 insertions(+)


# Experimenting with reproducible pipelines

## How reproduce experiments?

> The most exciting part of DVC is reproducibility.
>> Reproducibility is the time you are getting benefits out of DVC instead of spending time defining the ML pipelines.

> DVC tracks all the dependencies, which helps you iterate on ML models faster without thinking what was affected by your last change.
>> In order to track all the dependencies, DVC finds and reads ALL the DVC-files in a repository and builds a dependency graph (DAG) based on these files.

> This is one of the differences between DVC reproducibility and traditional Makefile-like build automation tools (Make, Maven, Ant, Rakefile etc). It was designed in such a way to localize specification of DAG nodes.
If you run repro on any created DVC-file from our repository, nothing happens because nothing was changed in the defined pipeline.

(c) dvc.org https://dvc.org/doc/tutorial/reproducibility

In [27]:
# Nothing to reproduce
!dvc repro

Stage 'data_load' didn't change, skipping                             core>
Stage 'feature_extraction' didn't change, skipping
Stage 'split_dataset' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


## Experiment 1: Add features



### Create new experiment branch

Before editing the code/featurization.py file, please create and checkout a new branch __ratio_features__

In [28]:
# create new branch

!git checkout -b exp1-ratio-features
!git branch

Переключено на новую ветку «exp1-ratio-features»
  dev
  dev-update-pipelines
  dvc-tutorial
* exp1-ratio-features
  master
  update-confusion-matrix
  update-dvc-tutorials
  update-software


### Update featurization.py

in file __featurization.py__  in function`get_features()` after line 

```python
    features = dataset.copy()
```

add lines:

```python
    features['sepal_length_to_sepal_width'] = features['sepal_length'] / features['sepal_width']
    features['petal_length_to_petal_width'] = features['petal_length'] / features['petal_width']
```

### Reproduce pipeline 

In [29]:
!dvc repro

Stage 'data_load' didn't change, skipping                             core>
Running stage 'feature_extraction' with command:
	python src/featurization.py --config=params.yaml
Updating lock file 'dvc.lock'                                         core>

Running stage 'split_dataset' with command:
	python src/split_dataset.py --config=params.yaml
Updating lock file 'dvc.lock'                                         core>

Running stage 'train' with command:
	python src/train.py --config=params.yaml
Updating lock file 'dvc.lock'                                         core>

Running stage 'evaluate' with command:
	python src/evaluate.py --config=params.yaml
Updating lock file 'dvc.lock'                                         core>

To track the changes with git, run:

	git add dvc.lock


In [30]:
# Check features used in this pipeline

import pandas as pd

features = pd.read_csv('data/iris_featurized.csv')
features.head()

,sepal_length,sepal_width,petal_length,petal_width,target,sepal_length_to_sepal_width,petal_length_to_petal_width
0,5.1,3.5,1.4,0.2,0,1.457143,7.0
1,4.9,3.0,1.4,0.2,0,1.633333,7.0
2,4.7,3.2,1.3,0.2,0,1.468750,6.5
3,4.6,3.1,1.5,0.2,0,1.483871,7.5
4,5.0,3.6,1.4,0.2,0,1.388889,7.0


In [31]:
!git status

На ветке exp1-ratio-features
Изменения, которые не в индексе для коммита:
  (используйте «git add <файл>…», чтобы добавить файл в индекс)
  (используйте «git checkout -- <файл>…», чтобы отменить изменения
   в рабочем каталоге)

	изменено:      dvc-3-automate-experiments.ipynb
	изменено:      dvc.lock
	изменено:      src/featurization.py

нет изменений добавленных для коммита
(используйте «git add» и/или «git commit -a»)


In [32]:
# Get difference with metric from previous pipeline
!dvc metrics diff --all

Path               Metric    Old      New      Change                 core>
data/metrics.json  f1_score  0.15385  0.15385  0.0


In [33]:
!git add .
!git commit -m "Experiment with new features"
!git tag -a "exp1_ratio_features" -m "Experiment with new features"

[exp1-ratio-features 05dd8e3] Experiment with new features
 3 files changed, 83 insertions(+), 76 deletions(-)


## Experiment 2: Tune Logistic Regression

### Create a new experiment branch

In [34]:
# create new branch for experiment

!git checkout -b exp2-tuning-logreg
!git branch

Переключено на новую ветку «exp2-tuning-logreg»
  dev
  dev-update-pipelines
  dvc-tutorial
  exp1-ratio-features
* exp2-tuning-logreg
  master
  update-confusion-matrix
  update-dvc-tutorials
  update-software


In [35]:
# Nothing to reproduce since code was checked out by `git checkout`
# and data files were checked out by `dvc checkout`
!dvc repro

Stage 'data_load' didn't change, skipping                             core>
Stage 'feature_extraction' didn't change, skipping
Stage 'split_dataset' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


### Tuning parameters

in file __train.py__ :

replace LogisticRegression params with:

```python
    clf = LogisticRegression(C=0.1, solver='newton-cg', multi_class='multinomial', max_iter=100)
```
__Note__: here we changed logistic regresssion hyperparameters: C  to 0.1


https://dvc.org/doc/tutorials/get-started/experiments#tuning-parameters

### Reproduce pipelines

In [36]:
# re-run pipeline 

!dvc repro

Stage 'data_load' didn't change, skipping                             core>
Stage 'feature_extraction' didn't change, skipping
Stage 'split_dataset' didn't change, skipping
Running stage 'train' with command:
	python src/train.py --config=params.yaml
Updating lock file 'dvc.lock'                                         core>

Running stage 'evaluate' with command:
	python src/evaluate.py --config=params.yaml
Updating lock file 'dvc.lock'                                         core>

To track the changes with git, run:

	git add dvc.lock


In [37]:
# Get difference with metric from previous pipeline
!cat data/metrics.json

{"f1_score": 1.0}

In [38]:
!dvc metrics show

	data/metrics.json:                                                   core>
		f1_score: 1.0


In [39]:
!dvc metrics diff --all

Path               Metric    Old      New    Change                   core>
data/metrics.json  f1_score  0.15385  1.0    0.84615


### Commit

In [40]:
%%bash

git add .
git commit -m "Tune model. LogisticRegression. C=0.1, solver=newton-cg"
git tag -a "exp2_tuning_logreg" -m "Tune model. LogisticRegression. C=0.1, solver=newton-cg"

[exp2-tuning-logreg c2e3cc6] Tune model. LogisticRegression. C=0.1, solver=newton-cg
 3 files changed, 33 insertions(+), 30 deletions(-)


## Experiment 3: Use SVM

### Create a new experiment branch

In [41]:
!git checkout -b exp3-svm
!git branch

M	dvc-3-automate-experiments.ipynb
Переключено на новую ветку «exp3-svm»
  dev
  dev-update-pipelines
  dvc-tutorial
  exp1-ratio-features
  exp2-tuning-logreg
* exp3-svm
  master
  update-confusion-matrix
  update-dvc-tutorials
  update-software


### Update train.py

in file __train.py__ replace line

```python
    clf = LogisticRegression(C=0.1, solver='newton-cg', multi_class='multinomial', max_iter=100)
```

with line

```python
    clf = SVC(C=0.01, kernel='linear', gamma='scale', degree=5)
```


### Reproduce pipeline 

In [42]:
!dvc repro

Stage 'data_load' didn't change, skipping                             core>
Stage 'feature_extraction' didn't change, skipping
Stage 'split_dataset' didn't change, skipping
Running stage 'train' with command:
	python src/train.py --config=params.yaml
Updating lock file 'dvc.lock'                                         core>

Running stage 'evaluate' with command:
	python src/evaluate.py --config=params.yaml
Updating lock file 'dvc.lock'                                         core>

To track the changes with git, run:

	git add dvc.lock


In [43]:
!dvc metrics show

	data/metrics.json:                                                   core>
		f1_score: 1.0


In [44]:
!git status

На ветке exp3-svm
Изменения, которые не в индексе для коммита:
  (используйте «git add <файл>…», чтобы добавить файл в индекс)
  (используйте «git checkout -- <файл>…», чтобы отменить изменения
   в рабочем каталоге)

	изменено:      dvc-3-automate-experiments.ipynb
	изменено:      dvc.lock
	изменено:      src/train.py

нет изменений добавленных для коммита
(используйте «git add» и/или «git commit -a»)


In [45]:
!git add .
!git commit -m "Experiment 3 with SVM estimator"
!git tag -a "exp3_svm" -m "Experiment 3 with SVM estimator"

[exp3-svm 301b6a4] Experiment 3 with SVM estimator
 3 files changed, 37 insertions(+), 33 deletions(-)


## Merge best experiment `dvc-tutorial ` branch

In [46]:
%%bash

git checkout dvc-tutorial 
git merge exp3_svm

Merge made by the 'recursive' strategy.
 dvc-3-automate-experiments.ipynb | 251 ++++++++++++++++++++-------------------
 dvc.lock                         |  24 ++--
 src/featurization.py             |   3 +
 src/train.py                     |   2 +-
 4 files changed, 147 insertions(+), 133 deletions(-)


Переключено на ветку «dvc-tutorial»


# Compare experiment

## Compare params 

In [47]:
# Get params diffs 

!dvc params diff

                                                                  core>

In [48]:
# Compare parameters with a specific commit, a tag or any revision

!dvc params diff --all

Path         Param                         Old                       Newre>
params.yaml  data_load.classes_names_path  data/classes.json         data/classes.json
params.yaml  data_load.dummy_param         dummy_value               dummy_value
params.yaml  data_load.raw_data_path       data/iris.csv             data/iris.csv
params.yaml  data_split.test_path          data/test.csv             data/test.csv
params.yaml  data_split.test_size          0.2                       0.2
params.yaml  data_split.train_path         data/train.csv            data/train.csv
params.yaml  evaluate.confusion_matrix     data/cm.csv               data/cm.csv
params.yaml  evaluate.metrics_file         data/metrics.json         data/metrics.json
params.yaml  featurize.features_path       data/iris_featurized.csv  data/iris_featurized.csv
params.yaml  featurize.target_column       target                    target
params.yaml  train.model_path              data/model.joblib         data/model.joblib


In [49]:
!dvc params diff --show-json --all

{"params.yaml": {"data_split.test_path": {"old": "data/test.csv", "new": "data/test.csv"}, "featurize.target_column": {"old": "target", "new": "target"}, "evaluate.confusion_matrix": {"old": "data/cm.csv", "new": "data/cm.csv"}, "train.model_path": {"old": "data/model.joblib", "new": "data/model.joblib"}, "data_load.classes_names_path": {"old": "data/classes.json", "new": "data/classes.json"}, "data_split.test_size": {"old": 0.2, "new": 0.2, "diff": 0.0}, "data_load.dummy_param": {"old": "dummy_value", "new": "dummy_value"}, "data_load.raw_data_path": {"old": "data/iris.csv", "new": "data/iris.csv"}, "data_split.train_path": {"old": "data/train.csv", "new": "data/train.csv"}, "evaluate.metrics_file": {"old": "data/metrics.json", "new": "data/metrics.json"}, "featurize.features_path": {"old": "data/iris_featurized.csv", "new": "data/iris_featurized.csv"}}}


In [50]:
!dvc params diff --show-md --all

| Path        | Param                        | Old                      | New                      |
|-------------|------------------------------|--------------------------|--------------------------|
| params.yaml | data_load.classes_names_path | data/classes.json        | data/classes.json        |
| params.yaml | data_load.dummy_param        | dummy_value              | dummy_value              |
| params.yaml | data_load.raw_data_path      | data/iris.csv            | data/iris.csv            |
| params.yaml | data_split.test_path         | data/test.csv            | data/test.csv            |
| params.yaml | data_split.test_size         | 0.2                      | 0.2                      |
| params.yaml | data_split.train_path        | data/train.csv           | data/train.csv           |
| params.yaml | evaluate.confusion_matrix    | data/cm.csv              | data/cm.csv              |
| params.yaml | evaluate.metrics_file        | data/metrics.json        | data/metrics.json

In [51]:
# To see the difference between two specific commits, both need to be specified:

!git log

commit f5aa4a3d0520d30f051d63331573d34b48a55c3e
Merge: 5138317 301b6a4
Author: Alex <alexckolosov@gmail.com>
Date:   Mon Oct 19 17:46:32 2020 +0900

    Merge tag 'exp3_svm' into dvc-tutorial
    
    Experiment 3 with SVM estimator

commit 301b6a44627d4c86536f7f6efdc33753c6c4d5ca
Author: Alex <alexckolosov@gmail.com>
Date:   Mon Oct 19 17:46:28 2020 +0900

    Experiment 3 with SVM estimator

commit c2e3cc69b18c99edd995cc851f1592d9008e431a
Author: Alex <alexckolosov@gmail.com>
Date:   Mon Oct 19 17:44:10 2020 +0900

    Tune model. LogisticRegression. C=0.1, solver=newton-cg

commit 05dd8e3f953cb5cdd11779455f34c0730adb749d
Author: Alex <alexckolosov@gmail.com>
Date:   Mon Oct 19 17:42:12 2020 +0900

    Experiment with new features

commit 5138317f18dcb8c64948181b865f8ce772b59b1e
Author: Alex <alexckolosov@gmail.com>
Date:   Mon Oct 19 17:40:36 2020 +0900

    Add stage evaluate

commit a6ba55df4d1d14d3fa0ee84f5a2705c26e260f5c
Author: Alex <alexckolosov@gmail.com>
Date:   Mon Oct 19 1

In [54]:
!dvc params diff f5aa4a3d0520d30f051d63331573d34b48a55c3e HEAD^

                                                                  core>

## Show metrics

In [55]:
# this pipeline metrics 

!dvc metrics show

	data/metrics.json:                                                   core>
		f1_score: 1.0


In [56]:
# show all commited pipelines metrics (all branch and tags)

!dvc metrics show -a -T

dvc-tutorial:                                                         core>
	data/metrics.json:
		f1_score: 1.0
exp1-ratio-features:
	data/metrics.json:
		f1_score: 0.15384615384615383
exp2-tuning-logreg:
	data/metrics.json:
		f1_score: 1.0
exp3-svm:
	data/metrics.json:
		f1_score: 1.0
exp1_ratio_features:
	data/metrics.json:
		f1_score: 0.15384615384615383
exp2_svm:
	data/metrics.json:
		f1_score: 1.0
exp2_tuning_logreg:
	data/metrics.json:
		f1_score: 1.0
exp3_svm:
	data/metrics.json:
		f1_score: 1.0


## Compare metrics (get differences)


In [57]:
!dvc metrics diff

                                                                  core>

In [58]:
# --all - list all metrics, even those without changes

!dvc metrics diff --all

Path               Metric    Old    New    Change                     core>
data/metrics.json  f1_score  1.0    1.0    0.0


* чтобы сравнить текущую метрики из текущего коммита и из другого, нужно указать другой (old) коммит:

In [60]:
# Compare old and new branches


!dvc metrics diff exp1-ratio-features exp3-svm

Path               Metric    Old      New    Change                   core>
data/metrics.json  f1_score  0.15385  1.0    0.84615


In [61]:
# Equivalent to `!dvc metrics diff exp1-ratio-features dvc-tutorial`, because dvc-tutorial - current branch

!dvc metrics diff exp1-ratio-features

Path               Metric    Old      New    Change                   core>
data/metrics.json  f1_score  0.15385  1.0    0.84615


In [64]:
!dvc metrics diff --show-md

| Path   | Metric   | Old   | New   | Change   |                      core>
|--------|----------|-------|-------|----------|



## Build Plots


In [65]:
from IPython.display import IFrame

### Show

In [66]:
!dvc plots show  --template confusion "data/cm.csv" -x actual -y predicted -o data/plots-show.html

file:///home/alex/Dev/Projects/tutorials/tutorials-dvc/dvc-3-automate-experiments/data/plots-show.html


In [73]:
IFrame(src='data/plots-show.html', width=400, height=300)

### Diff

In [69]:
# Build metircs plots for all 3 experiments
!dvc plots diff -t confusion -o data/plots-diff.html exp1-ratio-features exp3-svm -x predicted

file:///home/alex/Dev/Projects/tutorials/tutorials-dvc/dvc-3-automate-experiments/data/plots-diff.html


In [74]:
IFrame(src='data/plots-diff.html', width=400, height=300)